# Dependencies

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install jiwer
!pip install torchaudio
!pip install librosa

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-c7iop258
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /tmp/pip-req-build-c7iop258
  Resolved https://github.com/huggingface/datasets.git to commit 5f42139a2c5583a55d34a2f60d537f5fba285c28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for datasets: filename=datasets-2.21.1.dev0-py3-none-any.whl size=505185 sha256=143c60eea5c2210e16dab9e7aafe4cc19f8f0409eb3052ea22ac0805e450f305
  Stored in directory: /tmp/pip-ephem-wheel-cache-r4g6raun/wheels/57/f4/c4/53c677af89fec0ef3226c1e75a38367b37c2fa626f0544d3e4
Successfully built datasets
  Attempting uninstall: datasets
    Found existing installation: datasets 2.20.0
    Uninstalling datasets-2.20.0:
      Successfully uninstalled datasets-2.20.0
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip

In [4]:
!pip install dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.25
    Uninstalling SQLAlchemy-2.0.25:
      Successfully uninstalled SQLAlchemy-2.0.25
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.53 which is incompatible.


In [5]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys

In [5]:
from huggingface_hub import interpreter_login

# Perform a login to the Hugging Face Hub.
interpreter_login()




    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ·····································
Add token as git credential? (Y/n)  n


Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Kaggle dataset to hugging face dataset

In [5]:
import torchaudio
from tqdm import tqdm
from pathlib import Path

data = []

# Use tqdm to show progress as you process each audio file
for path in tqdm(Path("/kaggle/input/in-the-wild-audio-deepfake/release_in_the_wild").glob("**/*.wav"), desc="Processing audio files"):
    name = str(path).split('/')[-1].split('.')[0]
    
    try:
        # Attempt to load the audio file
        s = torchaudio.load(path)
        data.append({
            "path": str(path),
        })
    except Exception as e:
        # Handle any exceptions (e.g., corrupted files) and skip the file
        pass

# Now `data` contains the paths of all successfully loaded audio files


Processing audio files: 31779it [08:15, 64.16it/s]


In [6]:
df = pd.DataFrame(data)
df.head()

,path
0,/kaggle/input/in-the-wild-audio-deepfake/relea...
1,/kaggle/input/in-the-wild-audio-deepfake/relea...
2,/kaggle/input/in-the-wild-audio-deepfake/relea...
3,/kaggle/input/in-the-wild-audio-deepfake/relea...
4,/kaggle/input/in-the-wild-audio-deepfake/relea...


In [7]:
df["path"][0]

'/kaggle/input/in-the-wild-audio-deepfake/release_in_the_wild/fake/9693.wav'

In [8]:
str(df["path"][0])[-1]

'v'

In [9]:
data = pd.read_csv("/kaggle/input/in-the-wild-audio-deepfake/modified_meta.csv")
data.head()

,file,label
0,0.wav,fake
1,1.wav,fake
2,2.wav,fake
3,3.wav,fake
4,4.wav,real


In [10]:
data["file"][0]

'0.wav'

In [11]:
data.shape

(31779, 2)

In [12]:
data[data["file"]=="12187.wav"]["label"]

12187    real
Name: label, dtype: object

In [13]:
data["label"].value_counts()

label
real    19963
fake    11816
Name: count, dtype: int64

In [14]:
df['path'] = df['path'].astype(str)
df['file'] = df['path'].apply(lambda x: x.split('/')[-1])

# Merge the DataFrames based on the file number
df1 = pd.merge(df, data, on='file', how='inner')
df1.tail()

,path,file,label
31774,/kaggle/input/in-the-wild-audio-deepfake/relea...,12187.wav,real
31775,/kaggle/input/in-the-wild-audio-deepfake/relea...,15764.wav,real
31776,/kaggle/input/in-the-wild-audio-deepfake/relea...,11964.wav,real
31777,/kaggle/input/in-the-wild-audio-deepfake/relea...,8864.wav,real
31778,/kaggle/input/in-the-wild-audio-deepfake/relea...,12668.wav,real


In [15]:
df1.shape

(31779, 3)

In [16]:
df1.dropna(inplace=True)

In [17]:
df1.shape

(31779, 3)

In [18]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

# Randomly select a sample
idx = np.random.randint(0, len(df))
sample = df1.iloc[idx]
path = sample["path"]
label = sample["label"]

print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

# Load the audio file
speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()

# Resample the audio to 16,000 Hz
speech = librosa.resample(y=speech, orig_sr=sr, target_sr=16000)

# Play the audio
ipd.Audio(data=speech, autoplay=True, rate=16000)


ID Location: 25833
      Label: real



In [19]:
train_df, test_df,  = train_test_split(df1, test_size=0.2, random_state=101, stratify=df1["label"])

In [20]:
train_df.shape, test_df.shape

((25423, 3), (6356, 3))

In [21]:
train_df.head()

,path,file,label
29099,/kaggle/input/in-the-wild-audio-deepfake/relea...,1916.wav,real
30416,/kaggle/input/in-the-wild-audio-deepfake/relea...,23220.wav,real
3406,/kaggle/input/in-the-wild-audio-deepfake/relea...,26083.wav,fake
15968,/kaggle/input/in-the-wild-audio-deepfake/relea...,11705.wav,real
16868,/kaggle/input/in-the-wild-audio-deepfake/relea...,12515.wav,real


In [22]:
save_path = "/kaggle/working/"
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)


print(train_df.shape)
print(test_df.shape)

(25423, 3)
(6356, 3)


# Loading Dataset

In [5]:
# Loading the created dataset using datasets
from datasets import load_dataset


data_files = {
    "train": "/kaggle/input/iwa-testtrain/train.csv", 
    "validation": "/kaggle/input/iwa-testtrain/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print(train_dataset)
print(eval_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['path', 'file', 'label'],
    num_rows: 25423
})
Dataset({
    features: ['path', 'file', 'label'],
    num_rows: 6356
})


In [6]:
input_column = "path"
output_column = "label"

In [7]:
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 2 classes: ['fake', 'real']


In [8]:
from transformers import AutoConfig, Wav2Vec2Processor

In [6]:
model_name_or_path = "Anshul15/dfmodel_epoch2"
model_name_or_path1 = "facebook/wav2vec2-base"
pooling_mode = "mean"

In [10]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

config.json:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

In [11]:
print(f"Pooling mode set to: {config.pooling_mode}")

Pooling mode set to: mean


In [12]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path1,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The target sampling rate: 16000


In [13]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [23]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/25423 [00:00<?, ? examples/s]

2024-08-17 07:45:49.196457: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 07:45:49.196446: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 07:45:49.196446: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 07:45:49.196446: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 07:45:49.196527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory 

Map (num_proc=4):   0%|          | 0/6356 [00:00<?, ? examples/s]

2024-08-17 07:47:12.693410: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 07:47:12.693409: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 07:47:12.693473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 07:47:12.693481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 07:47:12.695726: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory

In [20]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


#### remove freeze weight 


In [15]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [16]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [27]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [28]:
is_regression = False

In [29]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [30]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

model.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

#### for full weight change comment next cell

In [31]:
model.freeze_feature_extractor()

In [32]:
from transformers import TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="/kaggle/working/dfmodel_epoch3",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=32,
    evaluation_strategy="steps",
    num_train_epochs=1.0,
    fp16=False,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    push_to_hub=True,
    load_best_model_at_end=True,
)

2024-08-17 07:48:41.255072: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 07:48:41.255150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 07:48:41.256619: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [33]:
from typing import Any, Dict, Union
import torch
from torch import nn
from transformers import Trainer

class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)

        # Compute loss in standard precision
        loss = self.compute_loss(model, inputs)

        # Ensure loss is a scalar by applying .mean() if needed
        if loss.dim() > 0:
            loss = loss.mean()

        # Gradient accumulation steps handling
        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        # Standard backpropagation
        loss.backward()

        return loss.detach()



In [34]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [ ]:
trainer.train()
#cac4d1683d6db21774b61c70382498cd50b0a60a

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy
10,0.029300,0.032700,0.995595


In [40]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Anshul15/dfmodel/commit/517c9675404ddefb4f370b5f908c590e68c5cfa0', commit_message='End of training', commit_description='', oid='517c9675404ddefb4f370b5f908c590e68c5cfa0', pr_url=None, pr_revision=None, pr_num=None)